In [1]:
import pyterrier as pt

if not pt.started():
    pt.init(tqdm="notebook")

dataset = pt.get_dataset("irds:beir/fiqa")

PyTerrier 0.10.0 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


Runtime indexing + cached/uncached test

In [2]:
from pathlib import Path

max_doc_id_length = 6
indexer = pt.IterDictIndexer(
    str(Path.cwd()),  # this will be ignored
    type=pt.index.IndexingType.MEMORY, meta={'docno': max_doc_id_length}
)

index_ref = indexer.index(dataset.get_corpus_iter(), fields=["text"])
bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25")

beir/fiqa documents:   0%|          | 0/57638 [00:00<?, ?it/s]

In [3]:
from pyterrier.measures import RR, nDCG, MAP
from experiment_utils.experiments_helper import time_fct

testset = pt.get_dataset("irds:beir/fiqa/test")

Runtime Uncached:

In [4]:
time_fct(pt.Experiment,
         [bm25],
         testset.get_topics(),
         testset.get_qrels(),
         eval_metrics=[RR @ 10, nDCG @ 10, MAP @ 100],
         )

11:52:41.364 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
Experiment took 17.715 seconds to execute.


,name,RR@10,nDCG@10,AP@100
0,BR(BM25),0.310271,0.252589,0.20864


Runtime Cached:

In [5]:
time_fct(pt.Experiment,
         [~bm25],
         testset.get_topics(),
         testset.get_qrels(),
         eval_metrics=[RR @ 10, nDCG @ 10, MAP @ 100],
         )

Experiment took 2.649 seconds to execute.


,name,RR@10,nDCG@10,AP@100
0,Cache(BR(BM25)),0.310271,0.252589,0.20864


Retrieve index from disk

In [6]:
index_path = "./sparse_index_fiqa"
# Load index to memory
index_disk = pt.IndexFactory.of(index_path, memory=True)

bm25_disk = pt.BatchRetrieve(index_disk, wmodel="BM25")

From disk uncached:

In [7]:
time_fct(pt.Experiment,
         [bm25_disk],
         testset.get_topics(),
         testset.get_qrels(),
         eval_metrics=[RR @ 10, nDCG @ 10, MAP @ 100],
         )

Experiment took 18.679 seconds to execute.


,name,RR@10,nDCG@10,AP@100
0,BR(BM25),0.310271,0.252589,0.20864


From disk cached:

In [8]:
time_fct(pt.Experiment,
         [~bm25_disk],
         testset.get_topics(),
         testset.get_qrels(),
         eval_metrics=[RR @ 10, nDCG @ 10, MAP @ 100],
         )

Experiment took 2.604 seconds to execute.


,name,RR@10,nDCG@10,AP@100
0,Cache(BR(BM25)),0.310271,0.252589,0.20864
